In [49]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_azure_ai.chat_models import AzureChatOpenAI

myendpoint = "https://nirav-mkv49s8o-westus3.cognitiveservices.azure.com/"
mydeployment_name = "gpt-4.1"
myapi_version = "2024-12-01-preview"

llm = AzureChatOpenAI(
    model_name=mydeployment_name,
    api_version=myapi_version,
    azure_endpoint=myendpoint
    # api_key=api_key,
)

# llm.invoke([{"role": "user", "content": "Hello, how are you?"}])

AIMessage(content='Hello! I’m just a language model, but I’m ready to help you. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 13, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_b9041e10b4', 'id': 'chatcmpl-D4hix7xCYbcyIonR0CgNPVwwkaZR9', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 's

In [72]:
from typing import TypedDict
from langgraph.graph import StateGraph
# from langgraph.annotations import Annotated
from langgraph.graph.message import add_messages, Annotated

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]} 

builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)

# builder.add_edge("START", "chatbot_node")
# builder.add_edge("chatbot_node", "END")

builder.set_entry_point("chatbot_node")

graph = builder.compile()


In [73]:
messages = {"role": "user", "content": "Hello, how are you?"}
graph.invoke({"messages": [messages]})

{'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='0a1458ee-1bc9-4b3c-a893-3b74eec5a4bb'),
  AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 13, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_b9041e10b4', 'id': 'chatcmpl-D4hrPl6oSfB1zhXCft0x8MtNsgDyW', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': '